# Agenda, day 2

1. Recap + Q&A
2. Magic methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? Often, it's easiest if we can use a data structure that's appropriate for the problem we're trying to solve. Objects allow us to create a data structure that is specifically designed to solve our problem, or to make it easier to think about our problem. Even if it's just a wrapper around strings, lists, tuples, dicts, etc., often the fact that an object has a different name can help us to solve our problems more easily.

The idea of object-oriented programming is: Create new data structures that are easier to use and think about than existing data structures.

- By putting these core data structures inside of an object, we can think at a higher level -- abstraction
- Methods (verbs) are defined on the class, which means that they are tightly bound to a particular type of data. Regular functions can be invoked with *any* type of value as an argument, although that might not be appropriate, and you can get an error. Methods are only available on the objects for which they actually exist. If you try to run a method on an object where it wasn't defined, you'll get an "attribute error," meaning that the name after the `.` doesn't exist.
- We define new "classes," data structures/types. A type is a class, and a class is a type -- at least when we're talking about them. The `type` function, which tells us what class something has, can only be used there. The `class` keyword can only be used when introducing a new data structure to Python.
- A class is also a factory for new objects of that type. If I have a `Person` class, then I can create new instances of `Person` by invoking the class, `Person()`. (Maybe I need to pass one or more arguments.) We often say that a class is a "factory object," because it creates new instances.
- The most important method in a class is `__init__` ("dunder init") whose job is to add new attributes to the object just after it's created, but before it's returned to the caller. `__init__` is invoked automatically by `__new__`, which we never rewrite, but which creates the new object. If you create 5 instances of `Person`, then `__init__` will be invoked 5 times, once for (and on) each instance.
- Each time we invoke a method, the first argument is automatically set to the instance itself. If I call `a.b()`, then the method `b` will get `a` as its first argument. However, the parameter into which `a` will be assigned is always called `self`. That parameter is automatically populated, but it is important.
- The whole idea of `self` is that it refers to the current instance. Other languages often have a special name, often `this`, which refers to the current object. Python is unusual in that `self` is a local variable, a parameter, just like any other local variable or parameter. It has the same priority, no magic needed, as any other local variable.
- When we assign to an attribute on `self`, we are adding an attribute to our object. (Unless it already exists, in which case we're updating the value for that attribute.) This storage on the object sticks around until the object goes away. If you set an attribute in `__init__` on `self`, that attribute exists on that object for the rest of the Python session, unless you delete the object or the attribute.
- In theory, you could assign all attributes to the object outside of `__init__`. However, it's easiest and best to assign attributes inside of `__init__`, so that we can keep track of things more easily. When you want to maintain someone else's class definition, it's easiest to do so if you have a clear list of all attributes.
- Most attributes are set based on the parameters, which get their values from arguments to the class. But you can set attributes without any relationship to those parameters, if you want.



In [1]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0

    def greet(self):
        return f'Hello, {self.name}!'

p = Person('Reuven', 46)        

In [2]:
p.greet() 

'Hello, Reuven!'

In [3]:
p.name

'Reuven'

In [4]:
p.name = 'whoever'

In [5]:
p.greet()

'Hello, whoever!'

# Memory management

In Python, we have two ways in which the language handles memory issues:

1. Reference counting -- when there are 0 references to an object (e.g., variable names, references from lists/dicts/tuples), then the object is immediately erased, and its memory is freed up.
2. Garbage collection -- if we have a circular data structure, then there will always be at least one reference to it, and it won't get released. Python occasionally looks for these and releases the "islands" of memory

When is an object removed? We see, if it's not a circular island in memory, then whenever the last variable or other reference goes away.

Practically speaking, this means that if you have a global variable that refers to an object, then the object will stick around until Python shuts down.

# Methods

Normally, methods are

    - defined on the class (in the class body, with `def`)
    - invoked on the instance

But actually, Python does some rewriting here! If I say

    an_instance.a_method()

Python rewrites this to be

    a_class.a_method(an_instance)

In [6]:
s = 'abcde'

s.upper()   # I'm invoking the "str.upper" method via the instance

'ABCDE'

In [7]:
# I can equivalently say:

str.upper(s) # this is what Python wrote line 3 in the above cell to be

'ABCDE'

Now you can see how `self` is populated! The method call is rewritten, and the first argument becomes the instance.

Weak references -- these allow a variable to refer to a value without modifying its reference count. In this way, you can refer to a value without stopping it from being garbage collected.

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on your instance of `Cellphone`. It will return a string saying, "Calling..." and print the number that it is calling.

In [8]:
class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

In [9]:
vars(c1)

{'number': '12345', 'model': 'iPhone'}

In [10]:
vars(c2)

{'number': '67890', 'model': 'Samsung'}

In [11]:
c1.number

'12345'

In [12]:
c2.number

'67890'

In [13]:
c1.model

'iPhone'

In [14]:
c2.model

'Samsung'

In [15]:
# let's now add the "call" method

class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model
    def call(self):
        return f'Calling {self.number} on your {self.model}...'

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

print(c1.call())
print(c2.call())

Calling 12345 on your iPhone...
Calling 67890 on your Samsung...


In [16]:
# let's now add the "call" method

class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model
    def call(self, other_number):
        return f'Calling {other_number} from your {self.model}, with a number of {self.number}...'

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

print(c1.call('1111'))
print(c2.call('22222'))

Calling 1111 from your iPhone, with a number of 12345...
Calling 22222 from your Samsung, with a number of 67890...


# Single Responsibility Principle

This is a big deal in the world of objects -- the idea is that one class should do one thing, not many things. If you find yourself loading lots of functionality onto a particular class, that might mean you need to divide it into multiple classes.

# Magic methods

When we perform certain operations in Python, the language looks for a method with a specific name that implements that operation. These methods all start and end with a double underscore, `__`. They are also known as "dunder methods," but many people call them "magic methods." You almost never want to call a magic method directly! Rather, let Python invoke the method on your behalf at the appropriate time.

The point of magic methods is that we can make our objects adhere to Python conventions and expectations, and behave much like the core builtin objects -- str, list, tuple, dict.

There are about 100 magic methods, and many of them are for complex parts of Python that we're not going to cover here. 

# Magic method: `__len__`

If you call `len` (a builtin function) on an object, then Python looks for a `__len__` method on that object, and invokes it. If there is no `__len__` method, then we get an error, indicating that the "object has no len".

It's very very very very tempting to just invoke `__len__` directly, rather than invoking the `len` function which will invoke `__len__` behind the scenes. But don't do it!